In [1]:
# import modules
import glob
import numpy as np
import pandas as pd
from astropy.time import Time
import cdflib
import datetime
from astropy.table import Table
import math
from scipy.optimize import curve_fit
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
import sys

In [2]:
# defined functions
def earth_radius(B):
    B = math.radians(B)  # converting into radians
    a = 6378.137  # Radius at sea level at equator
    b = 6356.752  # Radius at poles
    c = (a ** 2 * math.cos(B)) ** 2
    d = (b ** 2 * math.sin(B)) ** 2
    e = (a * math.cos(B)) ** 2
    f = (b * math.sin(B)) ** 2
    R = math.sqrt((c + d) / (e + f))
    return R


def objective(x, a, b):
    return a * np.exp(b * x)


def find_nearest_idx(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [3]:
# defined parameters
spatial_window = 5 # km
temporal_window = 15 # min
altitude_window = 100 # km

In [50]:
def objective(x, a, b):
    return a * np.exp(b * x)


# for radar_file in radar_list:
#     print(radar_file.split('/')[-1])
# for i in iter(range(0,4)):
radar_file = '../data/external/Madrigal/MillstoneHillISRadar/mlh200227i.004.hdf5'

    
radar = (Table.read(radar_file, path='Data/Table Layout')).to_pandas()
radar['dates'] = Time(radar[['ut1_unix', 'ut2_unix']].mean(axis=1), format='unix').datetime

radar_metadata = (Table.read(radar_file, path='Metadata/Experiment Parameters')).to_pandas()

names = np.array([x.decode() for x in radar_metadata['name']])
values = np.array([x.decode() for x in radar_metadata['value']])

radar_metadata = pd.DataFrame({'name': names,
                       'value': values})
radar_metadata.set_index('name', inplace=True)
radar_metadata = dict(pd.DataFrame.transpose(radar_metadata))

radar_lat = float(radar_metadata['instrument latitude'].value)
radar_lon = float(radar_metadata['instrument longitude'].value)
radar_lon = (((radar_lon + 180) % 360) - 180)

# gf = gf_ts


# # filter space
# gf = gf[((gf['Latitude'] <= radar_lat + spatial_window) & (gf['Latitude'] >= radar_lat - spatial_window))]
# gf = gf[((gf['Longitude'] <= radar_lon + spatial_window) & (gf['Longitude'] >= radar_lon - spatial_window))]

# if len(gf)>0:
#     # filter time
#     gf = gf[((gf['dates'] <= radar['dates'].max() + datetime.timedelta(hours=6)) & (gf['dates'] >= radar['dates'].min() - datetime.timedelta(hours=6)))]

#     radar = radar[((radar['dates'] <= gf['dates'].max() + datetime.timedelta(minutes=temporal_window)) & (radar['dates'] >= gf['dates'].min() - datetime.timedelta(minutes=temporal_window)))]
    
#     # save original
#     radar_old = radar
    
#     # black profile
    
#     radar_mean = pd.DataFrame({'gdalt': radar['gdalt'],
#                                'ne': radar['ne']})
    
#     radar_mean = radar_mean.groupby(['gdalt']).mean()

#     Xresampled = np.arange(int(radar_mean.index.min()),int(radar_mean.index.max()),10)

#     radar_mean = radar_mean.reindex(radar_mean.index.union(Xresampled)).interpolate('spline', order=1).loc[Xresampled]
    
    
#     # filter altitude
#     Re = earth_radius(radar_lat)
#     gf_alt = np.mean(gf['Radius'] * 0.001 - Re)

#     radar = radar[((radar['gdalt'] <= gf_alt + altitude_window) & (radar['gdalt'] >= gf_alt - altitude_window))]
    


#     if len(radar)>0:

#         #                 print(radar_file.split('/')[-1])
#         #                 print('GF', gf.dates.min(), gf.dates.max())
#         #                 print('RADAR', radar.dates.min(), radar.dates.max())
#         #                 print(radar_mean)
#         #                 print('--------------')

#         fig, ax = plt.subplots(figsize=(10,15))

#         recnos = np.unique(radar['recno'])
#         for rec in recnos:
#             radar_plot = radar_old[radar_old['recno'] == rec]
#             plt.plot(radar_plot['ne'], radar_plot['gdalt'] , label = np.unique(radar_plot['dates'])[0], alpha = 0.25)

#         radar_mean_window = radar_mean[((radar_mean.index <= gf_alt + altitude_window) & (radar_mean.index >= gf_alt - altitude_window))]
# #         radar_mean_window = radar_mean[((radar_mean.index <= 600) & (radar_mean.index >= 200))]

#         radar['nel'] = np.log10(radar['ne'])
#         radar_mean['nel'] = np.log10(radar_mean['ne'])
#         radar_old['nel'] = np.log10(radar_old['ne'])
#         radar_mean_window['nel'] = np.log10(radar_mean_window['ne'])



#         # filter nan values
#         radar_mean = radar_mean.dropna(subset=['nel'])
#         radar_mean_window = radar_mean_window.dropna(subset=['nel'])


#         plt.plot(radar_mean['ne'],radar_mean.index, color = 'k')
#         plt.title(radar_file.split('/')[-1])


#         x = radar_mean_window['nel'].values
#         y = radar_mean_window.index.values


#         # curve fit
#         popt, _ = curve_fit(objective, x, y, maxfev=10000)
#         a, b = popt
        

#         # define a sequence of inputs between the smallest and largest known inputs
#         x_line = np.arange(min(x), max(x), 10e-6)
#         # calculate the output for the range
#         y_line = objective(x_line, a, b)

#         idx = find_nearest_idx(y_line, gf_alt)
#         radar_alt = y_line[idx]
#         radar_nel = x_line[idx]

#         gf_ne = np.mean(gf['Relative_Ne'])

#         plt.plot(10**x_line, y_line ,'--', color='red', lw=2.5)


#         plt.scatter(gf_ne,gf_alt, marker = 'X', color = 'r')
#         plt.axhline(y=gf_alt+100, color='k', linestyle='-')
#         plt.axhline(y=gf_alt-100, color='k', linestyle='-')



#         plt.ylim(0,1000)




In [49]:
radar_metadata['instrument latitude'].value

'42.619'

name,instrument,instrument code(s),kind of data file,kindat code(s),start time,end time,Cedar file name,status description,instrument latitude,instrument longitude,instrument altitude,instrument category,instrument PI,instrument PI email
value,Millstone Hill UHF Zenith Antenna,32,Zenith single-pulse basic parameters,3420,2020-02-27 13:00:24 UT,2020-02-28 04:56:48 UT,mlh200227i.004.hdf5,Unknown,42.619,288.51,0.146,Incoherent Scatter Radars,Phil Erickson,perickson@haystack.mit.edu


In [41]:
dict(pd.DataFrame.transpose(radar_metadata))

{'instrument': value    Millstone Hill UHF Zenith Antenna
 Name: instrument, dtype: object,
 'instrument code(s)': value    32
 Name: instrument code(s), dtype: object,
 'kind of data file': value    Zenith single-pulse basic parameters
 Name: kind of data file, dtype: object,
 'kindat code(s)': value    3420
 Name: kindat code(s), dtype: object,
 'start time': value    2020-02-27 13:00:24 UT
 Name: start time, dtype: object,
 'end time': value    2020-02-28 04:56:48 UT
 Name: end time, dtype: object,
 'Cedar file name': value    mlh200227i.004.hdf5
 Name: Cedar file name, dtype: object,
 'status description': value    Unknown
 Name: status description, dtype: object,
 'instrument latitude': value    42.619
 Name: instrument latitude, dtype: object,
 'instrument longitude': value    288.51
 Name: instrument longitude, dtype: object,
 'instrument altitude': value    0.146
 Name: instrument altitude, dtype: object,
 'instrument category': value    Incoherent Scatter Radars
 Name: instrum